<h2><left>- Hello, This Notebook Contains Facebook Web-bot. </left></h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
     <ul>
    <li><h3><left>Contents:</left></h3></li>
    <ul>
        <li><a href="#import">Import</a></li>
        <li><a href="#bot">Facebook Bot</a></li>
        <li><a href="#str">Strategies</a></li>
        <li><a href="#pipline">Pipline</a></li>
        <li><a href="#run">Running</a></li> 
        <li><a href="#save">Saving</a></li> 
    </ul>
         </ul>
</div>
<hr>

<a id="import"></a>
<li><h3><left>- Import</left></h3>
<h5><left>- here we depends on selenium library.</left></h5>

In [1]:
# import needed libraries:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import numpy as np
import re

import json
import time

In [1]:
# initial values:
details_path = 'details.json'
website = 'facebook'
url = 'https://touch.facebook.com'
file_path = '31_8_2021.csv'
pages_path = 'pages.csv'
data_columns = ['page_id','text','time','reactions','url']
pages_columns =['name','id','followers']

In [8]:
# to get user-name and password of facebook account.
def get_user_details(file_path,website):
    with open(file_path) as jsonfile:
        config = json.load(jsonfile)
    return config[website]

<a id="bot"></a>
<h3><left>- This is abstract Facebook Bot.</left></h3>
<h5><left>- contains the basic processes that human can do in facebook, such as searching for page or clicking on post.</left></h5>

In [9]:
class FbCrawler:
    def __init__(self,browser= None):
        if browser is None:
            profile = webdriver.FirefoxProfile() 
            profile.set_preference("permissions.default.image", 2)
            # 1 - Allow all images
            # 2 - Block all images
            # 3 - Block 3rd party images
            self.browser = webdriver.Firefox(firefox_profile=profile)
        else:
            self.browser = browser
        self.comments_shares_pattern = r'(\d+)\s(\w+)(\d+)\s(\w+)'
        

    def start_scraping(self,url):
        try:    
            self.browser.get(url)
        except Exception as ex:
            print('some error in getting url!')
            
    def login_(self,userdetails):
        try: 
            email =  self.browser.find_element_by_id('m_login_email')
            password = self.browser.find_element_by_id('m_login_password')
            submit = self.browser.find_element_by_class_name('_2pie')

            email.send_keys(user_details['user_name'])
            WebDriverWait(self.browser, 5)
            password.send_keys(user_details['password'])
            submit.click()
            wait = WebDriverWait(self.browser, 5)
        except Exception as ex:
            print('some error in logging in!',ex)
            
    def complete_login(self):
        try:
            WebDriverWait(self.browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '._52jf')))
            search_field = self.browser.find_element_by_css_selector('._52jf')
            if search_field is not None and search_field.text in 'تسجيل الدخول بضغطة واحدة':
                not_now = self.browser.find_element_by_css_selector('._56bt')
                ok = self.browser.find_element_by_css_selector('._56bu')
                choise = np.random.randint(2)
                if choise is 0:
                    ok.click()
                else:
                    not_now.click()
        except Exception as ex:
            print("some error in complete login or the page is not required!",ex)
            
                
    def search_on_page(self,page_name):
        try:
            search_btn = self.browser.find_element_by_id('search_jewel')
            search_btn.click()
            WebDriverWait(self.browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main-search-input')))
            search_field = self.browser.find_element_by_css_selector('#main-search-input')
            search_field.clear()
            search_field.send_keys(page_name)
            WebDriverWait(self.browser, 5)
            search_field.send_keys(Keys.ENTER)
            WebDriverWait(self.browser, 5)
        except Exception as ex:
            print('some error in searching page!',ex)
    
    def click_on_pages_tap(self,delay=1):
        try: 
            WebDriverWait(self.browser, delay*5)
            pages_tap= self.browser.find_element_by_css_selector('._x0a:nth-child(6)')
            pages_tap.click()
        except Exception as ex:
            print('some error on click page tap!',ex)
            
    def choose_from_results(self,result_num,delay=1):
        try:
            WebDriverWait(self.browser, delay*5).until(EC.presence_of_element_located((By.CLASS_NAME, '_2rgt')))
            pages = self.browser.find_elements_by_class_name('_2rgt')
            pages[result_num].click()
        except Exception as ex:
            print('some error in choosing from results!',ex)
            raise ex
            

    def get_name(self,delay=1):
        name = ''
        try: 
            WebDriverWait(self.browser, delay*2).until(EC.presence_of_element_located((By.CLASS_NAME, '_59k')))
            name = self.browser.find_element_by_class_name('_59k').text
            print(name)
        except Exception as ex:
            print('some error in get page name!', ex)
        return name
        
    def get_posts_chunk(self,current):
        p = self.browser.find_elements_by_class_name("_rnk")
        return p[current:]
    
    def get_post(self,current):
        p = self.browser.find_elements_by_class_name("_rnk")
#         p = self.browser.find_elements_by_id("u_9_c_jl")
        return p[current]
    
    def get_post_reactions(self,delay=1):
        reactions_list = []
        try:
            WebDriverWait(self.browser, 5*delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, "._1g06")))
            reactions_area =  self.browser.find_element_by_css_selector("._1g06")  
            reactions_area.click()    
            
            WebDriverWait(self.browser, 3*delay).until(EC.presence_of_element_located((By.CLASS_NAME, '_5p-9')))
            reactions = self.browser.find_elements_by_class_name('_5p-9')
            for r in reactions:
                area_label = r.get_attribute('aria-label')
                reactions_list.append(area_label)
            wait = WebDriverWait(self.browser, 0.5*delay)  
            self.browser.execute_script("window.history.go(-1)")
        except Exception as ex:
            self.browser.execute_script("window.history.go(-1)")
            print('some error in get post reactions!',ex)
        return reactions_list
    
    def get_post_text(self,delay=1):
        text =[]
        try: 
            WebDriverWait(self.browser, 5*delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p')))
            ps = self.browser.find_elements_by_css_selector('p')
            for t in ps:
                text.append(t.text)
        except Exception as ex:
            print('some error in get post text!',ex)
            raise ex
        return '\n'.join(text)
    
    def get_post_date_time(self):
        date_time = ''
        try:
            date_time = self.browser.find_element_by_css_selector('abbr').text
        except Exception as ex:
            print('some error in get post date-time!',ex)
        return date_time
    
    def get_commentes_shares_count(self):
        counts = []
        try:
            counts = fb_crawler.browser.find_elements_by_css_selector('._1fnt')
            re.search(ptn,mm)
        except Exception as ex:
            print('some error in get post comments!',ex)
        return counts 
    
    def return_back(self):
        self.browser.execute_script("window.history.go(-1)")
        
    def scroll_down_by_value(self, value,start=0):
        self.browser.execute_script("window.scrollBy("+str(start)+","+str(value)+")")
    def scroll_up_by_value(self):
        self.browser.execute_script("window.scrollTo(document.body.scrollHeight, document.body.scrollHeight-500);")
          
    def scroll_down(self,start=0):
        self.browser.execute_script("window.scrollTo("+str(start)+", document.body.scrollHeight);")
    def scroll_up(self):
        self.browser.execute_script("window.scrollTo(200, 100);")
          
    def page_has_loaded(self):
        page_state = self.browser.execute_script('return document.readyState;')
        return page_state == 'complete'
    def get_current_url(self):
        return self.browser.current_url
    def clear_memory(self):
        self.browser.delete_all_cookies()

<a id="str"></a>
<h3><left>- This is Strategies Class.</left></h3>
<h5><left>- contains some of algorithms that simulate the human way of surfing Facebook.</left></h5>

In [14]:
class FbStartegies:
    
    def __init__(self,crawler,pipline):
        self.crawler = crawler
        self.pipline = pipline
        
    def open_page(self,page_name,delay=1,result_to_select=0):
        try:
            self.crawler.search_on_page(page_name)
            time.sleep(5*delay)
           self.crawler.click_on_pages_tap()
            page_name = self.crawler.get_name()
            page_id= self.pipline.add_to_pages(page_name)
            self.crawler.choose_from_results(result_to_select)

            print('page choosen: ',result_to_select+1,page_name)
            time.sleep(1*delay)
        except Exception as ex:
            print('some error in open page!',ex)
            raise ex
            
        return page_name
   
    def reopen_searched_page(self,result_to_select,delay=1):
        self.crawler.return_back()
        time.sleep(10*delay)
        
       self.crawler.choose_from_results(result_to_select)
        
    def get_page_posts(self,page_name,posts_num,start=0,delay=1,result_to_select=0,search_page =True,max_posts_num=60):
        crowled_posts = 0
        page_id =0
        try:
            if search_page:
                page_id = self.open_page(page_name,result_to_select=result_to_select)
            else:
                page_id = self.pipline.add_to_pages(page_name)
            
            try:
                i=len(self.crawler.get_posts_chunk(0))
                while i < posts_num + start:
                    p = self.crawler.get_posts_chunk(i)
                    i += len(p)
                    self.crawler.scroll_down()
                print(i)
                
            except Exception as ex:
                print('some exception in browsing data!',ex)
            self.crawler.scroll_up()
            WebDriverWait(self.crawler.browser, 5)
            time.sleep(3*delay)
            i= start
            while i < posts_num + start:
                try:
                    post = {}
#                     self.crawler.scroll_up()
                    WebDriverWait(self.crawler.browser, 2)
                    time.sleep(1*delay)
                    p = self.crawler.get_post(i)
                    self.crawler.scroll_down_by_value(575)
                    p.click()
                    
                    try:
                        post['text'] = self.crawler.get_post_text(delay=delay)
                        print(post['text'])
                        post['url'] = self.crawler.get_current_url()
                        post['time'] = self.crawler.get_post_date_time()
                        reactions = self.crawler.get_post_reactions(delay=delay)
                        post['reactions'] = ';'.join(reactions)
                        post['page_id'] = page_id
                        
                        self.pipline.add_to_data(post)
                        self.crawler.return_back()
                    except Exception as ex:
                        print('some exception:',ex)
                        self.crawler.return_back()

                    i+=1
                    crowled_posts +=1
#                     del post
#                     del p 
                    print(i)
                except Exception as ex:
                    i+=1
                    crowled_posts +=1
                    print('some exception:',ex)
        except Exception as ex:
            print('some error!',ex)

    

<a id="pipline"></a>
<h3><left>- This is Pipline Class.</left></h3>
<h5><left>- contains methods to save gained posts into .csv file using pandas.</left></h5>

In [15]:
import pandas as pd
class Pipeline:
    def __init__(self,save_path,pages_path,**kwargs):
        columns = None
        self.data_path = save_path
        if 'data' in kwargs:
            self.data = kwargs['data']
            
        if 'pages' in kwargs:
            self.pages = kwargs['pages']
        else: 
            if 'data_columns' in kwargs:
                self.data_columns = kwargs['data_columns']
                
            if 'pages_columns' in kwargs:
                self.pages_columns = kwargs['pages_columns']
                
            self.data = pd.DataFrame(columns=kwargs['data_columns'])
            self.pages = pd.DataFrame(columns= kwargs['pages_columns'])
                
    def add_to_data(self,row):
        row = pd.Series(row)
        self.data= self.data.append(row, ignore_index=True)
    
    def add_to_pages(self,page_name):
        page_id = 'error_id'
        try:
            page_id = self.pages.index[self.pages['name']==page_name].tolist() 
            if len(page_id) == 0:
                page_id = int(time.time())
                page = pd.Series({'name':page_name,'id':page_id})
                self.pages= self.pages.append(page,ignore_index = True)
                return page_id
            return page_id[0]
        except Exception as ex:
            print('some error in add page to data!', ex)
            
    def save_data(self):
        try:
            self.data.to_csv(self.data_path)
        except Exception as ex:
            print('some exception saving data')

<a id="run"></a>
<h3><left>- Running.</left></h3>
<h5><left>- here we create our pipline, run our bot, and finally start gaining posts form facebook.</left></h5>

In [16]:
pipline= Pipeline(file_path,pages_path,data_columns = data_columns,pages_columns=pages_columns)

In [17]:
# fb_crawler = FbCrawler(fb_crawler.browser)
fb_crawler = FbCrawler()
fb_st = FbStartegies(fb_crawler,pipline)

In [19]:
user_details = get_user_details(details_path,website)
fb_crawler.start_scraping(url=url)
fb_crawler.login_(user_details)
time.sleep(2)

In [ ]:
# the facebook bot need parameters:
# page_name, how many post we want it to gain, where do we want it start gaining.
page_name = "يوميات قذيفة هاون في دمشق"
fb_st.get_page_posts(page_name=page_name,posts_num=50,start=0,
                     search_page=True,max_posts_num=50,delay =1)

In [21]:
pipline.data

,page_id,text,time,reactions,url
0,,لتوليد الكهرباء.. دراسة للاستفادة من غاز الميث...,٧ دقائق,,https://m.facebook.com/story.php?story_fbid=38...
1,,مجلس التعليم العالي: تسوية أوضاع الطلاب المستن...,٣٨ دقيقة,تفاعل ‏٥١٩‏ من الأشخاص مع هذا المنشور;تفاعل ‏٥...,https://m.facebook.com/story.php?story_fbid=38...
2,,لباس الرسمي الجديد للمنتخب السوري.. عمر خريبين...,ساعة واحدة,تفاعل ‏٩٢٨‏ من الأشخاص مع هذا المنشور;تفاعل ‏٩...,https://m.facebook.com/story.php?story_fbid=38...
3,,مدرسة البشائر العربية الخاصة\nالتسجيل مستمر لل...,ساعتان,تفاعل ‏٢٦٥‏ من الأشخاص مع هذا المنشور;تفاعل ‏٢...,https://m.facebook.com/story.php?story_fbid=38...
4,,تقدير كلفة تنظيف شاطئ بانياس من الفيول بـ3 ملا...,٣ ساعات,تفاعل ‏٢٫٤ ألف‏ من الأشخاص مع هذا المنشور;تفاع...,https://m.facebook.com/story.php?story_fbid=38...
5,,برأيكن بشو ممكن ترتبط السيارة بيوم السبت بالمت...,٤ ساعات,تفاعل ‏٥٥٦‏ من الأشخاص مع هذا المنشور;تفاعل ‏٥...,https://m.facebook.com/story.php?story_fbid=38...
6,,ازدياد بعدد العمليات و1500 ولادة طبيعية وقيصري...,٥ ساعات,تفاعل ‏٢٫١ ألف‏ من الأشخاص مع هذا المنشور;تفاع...,https://m.facebook.com/story.php?story_fbid=38...
7,,#مؤسسة_أوغاريت _للسياحة_والسفر ✈️✈️\nالأسعار ا...,٦ ساعات,تفاعل ‏٢٫٦ ألف‏ من الأشخاص مع هذا المنشور;تفاع...,https://m.facebook.com/story.php?story_fbid=38...
8,,إعصار محطم للأعصاب يضرب ولاية أمريكية.. إليك ا...,٦ ساعات,تفاعل ‏٣٢٣‏ من الأشخاص مع هذا المنشور;تفاعل ‏٣...,https://m.facebook.com/story.php?story_fbid=38...
9,,ارتفاع أسعار الموبايلات 300 %.. وبائعون: المور...,٦ ساعات,تفاعل ‏٣٫٢ ألف‏ من الأشخاص مع هذا المنشور;تفاع...,https://m.facebook.com/story.php?story_fbid=38...


<a id="save"></a>
<h3><left>- Saving.</left></h3>
<h5><left>- save posts and pages.</left></h5>

In [ ]:
pipline.save_data()
pipline.save_pages()

<h2><center>- The End -</center></h2>